<a href="https://colab.research.google.com/github/Saifali111/Git-demo/blob/main/Research_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.2 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import mne
from os.path import exists
from sklearn import preprocessing

# Define the number of healthy and MDD (Major Depressive Disorder) cases
H_num = 30  # Number of healthy cases
MDD_num = 34  # Number of MDD cases

# Initialize lists to store data and labels
v = []
Data = []
Label = []

# Define the categories for healthy and MDD cases
HorMDD = ['H', 'MDD']
# Define the eye states: EC (eyes closed), EO (eyes open), and TASK (P300 data)
eye_states = ['EC', 'EO', 'TASK']

# Directory where the .edf files are located)
directory = '/content/drive/My Drive/MDD-dataset/EDF_Data'  # Need to change to your own file path
#directory = '/Users/saifali/Desktop/Machine Learning/Research Project/Data/'


# Iterate over each category (healthy and MDD)
for x in HorMDD:
    # Set label based on category
    label_value = 0 if x == 'H' else 1
    num_subjects = H_num if x == 'H' else MDD_num

    # Process each subject
    for i in range(1, num_subjects + 1):
        # Iterate over each eye state
        for state in eye_states:
            # Construct the file path
            file = f"{directory}/{x} S{i} {state}.edf"

            # Check if the file exists
            if exists(file):
                # Read the EEG data from the .edf file
                data = mne.io.read_raw_edf(file, preload=True)
                raw_data = data.get_data()
                num_rows, num_cols = raw_data.shape

                # Ensure the number of rows is correct
                if num_rows > 19:
                    print(f"Extra channels in {file}: {num_rows}")
                    raw_data = np.delete(raw_data, 19, 0)

                # Ensure the data length is sufficient
                if raw_data.shape[1] < 61440:
                    print(f"File skipped due to insufficient length: {file}")
                    v.append(file)
                    continue

                # Trim or pad the data to a consistent length
                trimmed_data = raw_data[:, :61440]  # Trim if longer
                if trimmed_data.shape[1] < 61440:
                    padding = np.zeros((trimmed_data.shape[0], 61440 - trimmed_data.shape[1]))
                    trimmed_data = np.concatenate((trimmed_data, padding), axis=1)  # Pad if shorter

                # Append data and label
                Data.append(trimmed_data)
                Label.append(label_value)

                # Save the processed data and labels
                np.save(f"{directory}/Data/{x} S{i} {state}.npy", trimmed_data)
                np.save(f"{directory}/Label/{x} S{i} {state}.npy", label_value)
            else:
                print(f"File not found: {file}")

# Convert Data to an array with object dtype, if lengths vary
Data = np.array(Data, dtype=object)  # Handles any residual variations in shape
Label = np.asarray(Label)

Extracting EDF parameters from /content/drive/My Drive/MDD-dataset/EDF_Data/H S1 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 76799  =      0.000 ...   299.996 secs...
Extra channels in /content/drive/My Drive/MDD-dataset/EDF_Data/H S1 EC.edf: 22
Extracting EDF parameters from /content/drive/My Drive/MDD-dataset/EDF_Data/H S1 EO.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 89855  =      0.000 ...   350.996 secs...
Extra channels in /content/drive/My Drive/MDD-dataset/EDF_Data/H S1 EO.edf: 22
Extracting EDF parameters from /content/drive/My Drive/MDD-dataset/EDF_Data/H S1 TASK.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 154879  =      0.000 ...   604.996 secs...
Extra channels in /content/drive/My Drive/MDD-dataset/EDF_Data/H S1 TASK.edf: 22
Extracting EDF parameters from /content/drive/My Drive/MDD-dataset/E

In [1]:
# Check for CUDA availability.
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
torch.cuda.empty_cache()

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader


# Define constants for model configuration
output_directory = '/output'  # Directory to save output files
input_shape = [19, 61440]  # Input shape: 19 channels and 4-minute records
nb_classes = 2  # Number of classes: MDD or Healthy
nb_filters = 64  # Number of filters in Conv layers
verbose = True  # Verbose output during training
use_residual = True  # Use residual connections
use_bottleneck = True  # Use bottleneck layers
depth = 6  # Depth of the model
kernel_size = 41 - 1  # Kernel size for Conv layers
callbacks = None  # Callbacks for training
batch_size = 4  # Batch size for training
mini_batch_size = 32  # Mini-batch size for training
bottleneck_size = 57  # Bottleneck size
nb_epochs = 50  # Number of epochs for training
learning_rate = 0.01

# Define the Inception module
class InceptionModule(nn.Module):
    def __init__(self, in_channels, bottleneck_size, kernel_size, nb_filters):
        super(InceptionModule, self).__init__()
        self.use_bottleneck = use_bottleneck

        if self.use_bottleneck and in_channels > 1:
            self.bottleneck = nn.Conv1d(in_channels, bottleneck_size, kernel_size=1, padding='same', bias=False)
        else:
            self.bottleneck = nn.Identity()

        self.conv1 = nn.Conv1d(bottleneck_size, nb_filters, kernel_size=kernel_size // 2, padding='same', bias=False)
        self.conv2 = nn.Conv1d(bottleneck_size, nb_filters, kernel_size=kernel_size // 4, padding='same', bias=False)
        self.conv3 = nn.Conv1d(bottleneck_size, nb_filters, kernel_size=kernel_size // 8, padding='same', bias=False)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv1d(bottleneck_size, nb_filters, kernel_size=1, padding='same', bias=False)

        self.batch_norm = nn.BatchNorm1d(nb_filters * 4)

    def forward(self, x):
        x = self.bottleneck(x)
        conv1 = self.conv1(x)
        conv2 = self.conv2(x)
        conv3 = self.conv3(x)
        maxpool = self.conv4(self.maxpool(x))
        out = torch.cat([conv1, conv2, conv3, maxpool], dim=1)
        out = self.batch_norm(out)
        return F.relu(out)

# Define the shortcut layer for residual connections
class ShortcutLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ShortcutLayer, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=1, padding='same', bias=False)
        self.batch_norm = nn.BatchNorm1d(out_channels)

    def forward(self, input_tensor, out_tensor):
        shortcut = self.conv(input_tensor)
        shortcut = self.batch_norm(shortcut)
        out = shortcut + out_tensor
        return F.relu(out)

# Build the model
class InceptionModel(nn.Module):
    def __init__(self, input_shape, nb_classes, depth, bottleneck_size, kernel_size, nb_filters):
        super(InceptionModel, self).__init__()
        self.depth = depth
        self.use_residual = use_residual
        self.inception_modules = nn.ModuleList()
        in_channels = input_shape[0]

        for d in range(depth):
            self.inception_modules.append(InceptionModule(in_channels, bottleneck_size, kernel_size, nb_filters))
            in_channels = nb_filters * 4

        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(in_channels, nb_classes)

    def forward(self, x):
        input_res = x
        for d in range(self.depth):
            x = self.inception_modules[d](x)
            if self.use_residual and d % 3 == 2:
                x = ShortcutLayer(input_res.size(1), x.size(1)).to(x.device)(input_res, x)
                input_res = x

        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

# Move the model to the GPU
model = InceptionModel(input_shape, nb_classes, depth, bottleneck_size, kernel_size, nb_filters).to(device)

# Print the model summary
print(model)

# Define the optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
criterion = nn.MSELoss()

# Assuming X_train and y_train are numpy arrays
X_train = np.random.rand(100, *input_shape).astype(np.float32)
y_train = np.random.randint(0, 2, size=(100,)).astype(np.int64)

# Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(nb_epochs):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, F.one_hot(y_batch, num_classes=nb_classes).float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        # accuracy calculator
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    # Calculate accuracy
    accuracy = (correct / total) * 100

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{nb_epochs}, Loss: {epoch_loss / len(train_loader)}, Accuracy :{accuracy}")

InceptionModel(
  (inception_modules): ModuleList(
    (0): InceptionModule(
      (bottleneck): Conv1d(19, 57, kernel_size=(1,), stride=(1,), padding=same, bias=False)
      (conv1): Conv1d(57, 64, kernel_size=(20,), stride=(1,), padding=same, bias=False)
      (conv2): Conv1d(57, 64, kernel_size=(10,), stride=(1,), padding=same, bias=False)
      (conv3): Conv1d(57, 64, kernel_size=(5,), stride=(1,), padding=same, bias=False)
      (maxpool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv4): Conv1d(57, 64, kernel_size=(1,), stride=(1,), padding=same, bias=False)
      (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1-5): 5 x InceptionModule(
      (bottleneck): Conv1d(256, 57, kernel_size=(1,), stride=(1,), padding=same, bias=False)
      (conv1): Conv1d(57, 64, kernel_size=(20,), stride=(1,), padding=same, bias=False)
      (conv2): Conv1d(57, 64, kernel_size=(10,), stride=(1,), padding

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:370: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv1d(


Epoch 0/50, Loss: 883.3574112272263
Epoch 10/50, Loss: 5.203097133636475
Epoch 20/50, Loss: 3.338097958564758
Epoch 30/50, Loss: 3.1272587680816653
Epoch 40/50, Loss: 2.178376097679138
